In [7]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
model = keras.models.load_model('face-musk-model.h5')
import cv2

cap = cv2.VideoCapture(0)

#Check if camera was opened correctly
if not (cap.isOpened()):
    print("Could not open video device")

while(True):
    
    ret, frame = cap.read()

    frame = cv2.resize(frame, (227, 227))
    frame = frame.reshape(-1, 227, 227, 3)


    prediction = model.predict(frame)
    hasMusk = prediction[0][0]
    noMusk = prediction[0][1]

    if hasMusk > noMusk:
        # draw a box around the face
        cv2.rectangle(frame, (0, 0), (227, 227), (0, 255, 0), 2)
        # draw the predicted class and associated probability
        text = "{}: {:.2f}%".format("Has Mask", hasMusk * 100)
        cv2.putText(frame, text, (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        # draw a box around the face
        cv2.rectangle(frame, (0, 0), (227, 227), (0, 0, 255), 2)
        # draw the predicted class and associated probability
        text = "{}: {:.2f}%".format("No Mask", noMusk * 100)
        cv2.putText(frame, text, (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)



  
   

    cv2.imshow("preview",np.squeeze(frame))
   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 132ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step


2023-04-04 14:33:50.359240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==============================] - 0s 14ms/step
Mask
1.0
1/1 [==============================] - 0s 11ms/step
Mask
1.0
1/1 [==============================] - 0s 10ms/step
Mask
1.0
1/1 [==========